# capture cell 'code'

In [5]:
# https://stackoverflow.com/questions/33508377/read-cell-content-in-an-ipython-notebook
from IPython.core.magic import  (
    Magics, magics_class, cell_magic, line_magic
)

@magics_class
class CellsStorer(Magics):

    def __init__(self, shell=None,  **kwargs):
        super().__init__(shell=shell, **kwargs)
        self._store = dict()
        shell.user_ns['__store'] = self._store

    @cell_magic
    def cell_store(self, line, cell):
        """store the cell in the store"""
        key = line.strip()
        self._store[key] = cell

    @line_magic
    def show_store(self, line):
        """show all recorded statements"""
        print(self._store)

## use ipython load_ext mechanisme here if distributed
get_ipython().register_magics(CellsStorer)

In [7]:
%%cell_store py
import os

In [9]:
print(__store)

{'py': 'import os'}


# capture cell output

In [13]:
%%capture cap --no-stderr
print ('stuff')
#print ("[%s]" % cap.stdout)
with open('output.txt', 'w') as f:
    f.write(cap.stdout)

%%capture cell23
# Headline1
- bullet1
- bullet2

In [17]:
cap.cell23

AttributeError: 'CapturedIO' object has no attribute 'cell23'

# parse notebook itself
## and test markdown export

In [73]:
print("testing html escaping < & >")

testing html escaping < & >


In [25]:
import pandas as pd
df = pd.DataFrame(["A", "b", "cc"], [1, 2, 3])
df

,0
1,A
2,b
3,cc


In [74]:
from pygments import highlight
from pygments.lexers import get_lexer_by_name, get_lexer_for_filename
from pygments.formatters import HtmlFormatter
from html import escape as html_escape
import markdown

class NotebookConverter():
    def __init__(self, filename):
        import json
        with open(filename) as fp:
            self.book = json.load(fp)
    
    def to_html(self):
        html = []
        for cell in self.book["cells"]:
            if cell["cell_type"] == "markdown":
                html.append(markdown.markdown(" ".join(cell["source"])))
            if cell["cell_type"] == "code":
                html.append(self.python_to_html(" ".join(cell["source"])))
            if cell.get("outputs"):
                for output in cell["outputs"]:
                    if "text/html" in output.get("data", {}):
                        html.append( " ".join(output["data"]["text/html"]) )
                    if "text" in output:
                        html.append( "<pre>%s</pre>" % html_escape(" ".join(output["text"])) )
                    #else:
                    #    print("------", output)
        return "\n".join("<!-- cell #%s -->\n%s\n" % t for t in enumerate(html))
    
    @classmethod
    def python_to_html(cls, code):
        lexer = get_lexer_by_name("python")
        formatter = HtmlFormatter()
        return highlight(code, lexer, formatter)
    
print(NotebookConverter("./test-cell-output.ipynb").to_html())

<!-- cell #0 -->
<h1>capture cell 'code'</h1>

<!-- cell #1 -->
<div class="highlight"><pre><span></span><span class="c1"># https://stackoverflow.com/questions/33508377/read-cell-content-in-an-ipython-notebook</span>
 <span class="kn">from</span> <span class="nn">IPython.core.magic</span> <span class="kn">import</span>  <span class="p">(</span>
     <span class="n">Magics</span><span class="p">,</span> <span class="n">magics_class</span><span class="p">,</span> <span class="n">cell_magic</span><span class="p">,</span> <span class="n">line_magic</span>
 <span class="p">)</span>
 
 <span class="nd">@magics_class</span>
 <span class="k">class</span> <span class="nc">CellsStorer</span><span class="p">(</span><span class="n">Magics</span><span class="p">):</span>
 
     <span class="k">def</span> <span class="fm">__init__</span><span class="p">(</span><span class="bp">self</span><span class="p">,</span> <span class="n">shell</span><span class="o">=</span><span class="bp">None</span><span cl